# Использование ipywidgets в jupyter notebook

Виджеты в jupyter notebook - это по сути добавление к интерфейсу тетрадок. Они привносят интерактив, могут использоваться для хранения и передачи информации. С их помощью можно превратить тетрадку в нечто понятное простому пользователю и использовать как dashboard или простой аналитический инструмент.

Вот что вы увидите:

* как установить нужные библиотеки;
* как использовать виджеты;
* основные виды виджетов;
* как можно объединять виджеты и для чего они могут использоваться;
* пример того, как можно сделать виджеты, позволяющие простому человеку с помощью элементов интерейса проанализировать данные, построить визуализацию и попробовать машинное обучение;
* небольшой бонус с рассказом о расширенных возможностях tqdm;

In [3]:
1

1

## Установка

Установка сама по себе не является сложной: это можно сделать через pip:

```python
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

Или через Anaconda:

```python
conda install -c conda-forge ipywidgets
```

Стоит также установить widgetsnbextension.

Проблема бывает с тем, что виджеты перестают работать. Обычно помогает установка последних версий. Причём обновление через Anacona Navigator может не помочь, поскольку при установке мог использоваться канал, в котором нет последней версии. Так что при возникновении проблем стоит сразу проверить это.

In [4]:
#Импорт библиотек
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from IPython.display import display
from IPython.display import clear_output

## Основные способы использования виджетов

Во-первых, можно просто отобразить виджет сам по себе.

In [5]:
widgets.IntSlider()

IntSlider(value=0)

Но это просто создаёт слайдер, с которым можно поиграться, а пользы от него нет. Следующим шагом является создание переменной с виджетом

In [6]:
w = widgets.IntSlider()
w

IntSlider(value=0)

И теперь мы можем получить значение виджета и использовать для чего-нибудь.

In [7]:
print(w.value, w.value ** 2)

0 0


Другой вариант - отображение виджета с помощью функции IPython display, но это работает точно также как и сам вызов переменной.

In [8]:
display(w)

IntSlider(value=0)

Вы передвигали слайдер в прошлой ячейке? Если нет - попробуйте. Можно заметить, что слайдер двигается в обеих ячейках, где была использована переменная w. Это нормальное поведение - у нас есть лишь один объект на back-end, так что получаются синхронизированные объекты на front-end.

После того как в виджете было установлено некое значение, виджет можно закрыть командой `w.close()`.

Пока мы видели простые виджеты, значения которых можно было использовать в дальнейшем. Если же хочется, чтобы с выбранным значением что-то произошло сразу, стоит использовать `interact`. `interact` позволяет вызывать функцию, в которую передаётся значение виджета. При изменении значения виджета функция будет динамически показывать обновлённый результат.

In [9]:
def f(x):
    return x, str(x) * 2

In [10]:
interact(f, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-interact',))

Замечу, что в `interact` обязательно передавать начальное значение, поскольку это напрямую определяет тип виджета. Если передать int или float, то будет слайдер с соответствующими данными.

Кроме того можно передать:

* текст для получения поля ввода текста;
* список/словарь значений для выпадающего списка;
* булевое значение для чекбокса

In [11]:
interact(f, x='Текст');
interact(f, x=['лопата', 'солнце']);
interact(f, x=True);

interactive(children=(Text(value='Текст', description='x'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Dropdown(description='x', options=('лопата', 'солнце'), value='лопата'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

Кому-то может быть интересна возможность использовать `interact` в качестве декоратора. В таком случае функцию можно не передавать в `interact`.

In [12]:
@interact(x=1.0)
def g(x):
    return x

interactive(children=(FloatSlider(value=1.0, description='x', max=3.0, min=-1.0), Output()), _dom_classes=('widget-interact',))

Наконец, есть такая интересная штука как `interactive`. По сути это является комбинацией двух предыдущих спообов: мы задаем функцию, а потом можем использовать как сами результаты, так и значения отдельных виджетов.

По умолчанию `interactive` показывает только сами виджеты, но не результат функции. Чтобы увидеть результат, надо явно его отображать.

Замечу, что можно использовать больше одной переменной и получить больше одного виджета.

In [13]:
def f(x, y, z):
    if z == 'Сумма':
        display(x + y)
        return x + y
    else:
        display(x * y)
        return x * y

In [14]:
w = interactive(f, x=1, y=2.5, z=['Сумма', 'Произведение'])

In [15]:
w

interactive(children=(IntSlider(value=1, description='x', max=3, min=-1), FloatSlider(value=2.5, description='y', max=7.5, min=-2.5), Dropdown(description='z', options=('Сумма', 'Произведение'), value='Сумма'), Output()), _dom_classes=('widget-interact',))

In [16]:
print(w.kwargs)
print(w.result)

{'x': 1, 'y': 2.5, 'z': 'Сумма'}
3.5


## Применение виджетов

До сих пор я показывал отдельные виджеты и простые примеры их использования, но с ними можно делать много интересных вещей. Просто перечислять их - довольно нудно, поэтому я буду приводить примеры и на их основе объяснять возможности виджетов.

### Динамически изменяющиеся графики

Мы уже видели, что в `interact` можно передавать функцию, результат которой будет обновляться при изменении параметров. Интереснее посмотреть это на примере построения графиков. Я хочу сделать график, который будет зависеть от задаваемого параметра, а также предложить возможность выбирать цвет графика.

Для начала нужно создать виджет для выбора цвета. Вообще говоря виджет можно передавать в функцию как значение параметра, но это громоздко, поэтому удобнее создать виджет, а потом передавать его в виде переменной.

Виджет имеет следующие параметры:
* concise - не показывает название выбранного цвета, если True;
* description - описание, этот параметр есть у всех виджетов;
* value - значение по умолчанию, опять же есть почти у всех виджетов;
* disabled - возможность изменения значения виджета;

In [17]:
pick_color = widgets.ColorPicker(
    concise=False,
    description='Цвет линии:',
    value='teal',
    disabled=False
)

Теперь сам интерактивный виджет. Функция для построения графика будет иметь 2 параметра:
* n - просто некий параметр, который будет изменять функцию;
* c - цвет линии;

Далее использую `interact`:
* для n я передаю список значений, которые можно выбрать. В функции значение по умолчанию 3, именно оно будет выбрано по умолчанию;
* для цвета значением будет переменная с виджетом для выбора цвета. Цвет можно выбирать через диалоговое окошко или напрямую вводить название или код в поле. Стоит заметить, что значением по умолчанию является не orange, а teal, поскольку значение в виджете имеет более высокий приоритет;

In [18]:
def h(n=3, c='orange'):
    x = np.linspace(0, 5, 10)
    plt.plot(x, x ** (n / (x + 1)) + x ** (x / n) / (n ** 2) + (n - x) ** 2 - x * np.log(n) + n * np.exp(x / n),  color=c)
    plt.show()

interact(h, n=[i for i in range(2, 6)], c=pick_color);

interactive(children=(Dropdown(description='n', index=1, options=(2, 3, 4, 5), value=3), ColorPicker(value='teal', description='Цвет линии:'), Output()), _dom_classes=('widget-interact',))

### Действие при нажатии на кнопку

Виджеты позволяют делать вызов функции после определённых событий:

* При нажатии на кнопку;
* При завершении ввода текста;
* По мере изменения значения (обычно используется для слайдеров);

Кроме того можно связать виджеты - чтобы при изменении значения одного виджета изменялось значение другого виджета.

Рассмотрим пример: пользователь задаёт длины сторон прямоугольника и хочет узнать его площадь. Для начала я задаю 2 виджета с соответсвующими названиями. Далее ограничения значений можно задать минимальное и максимальное значение, а также шаг, с которым будут изменяться значения.

In [19]:
x = widgets.IntSlider(min=2, max=40.0, step=2, description='Высота')
y = widgets.FloatSlider(min=2, max=80.0, step=0.5, description='Ширина', value=5.0)

Добавим ограничение - высота прямоугольника не может быть больше его ширины. Для этого используется `observe` - при каждом изменении объекта мгновенно вызывается функция. А в функции мы будет приравнивать максимальное значение виджета x к текущему значению виджета y.

In [20]:
def update_x_range(*args):
    x.max = y.value
    
y.observe(update_x_range, 'value')

Теперь добавим кнопку, при нажатии на которую будет происходить вычисление. Для этого создаём соответствующий объект с подходящим названием. И здесь в первый раз используется настройка внешнего вида виджетов. layout позволяет настраивать css объектов. Большинство параметров взять из самого CSS, но есть и различия, почитать подробнее можно [тут](#http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Styling.html). В данном случае я просто задаю ширину кнопки.

При нажатии на кнопку будет показана площадь прямоугольника, кнопка и слайдеры станут неактивными.

In [21]:
button1 = widgets.Button(description="Нажмите, чтобы посчитать площадь прямоугольника!", layout=widgets.Layout(width='40%'))

def on_button_clicked(b):
    print('Площадь прямоугольника: {}.'.format(x.value * y.value))
    button1.disabled = True
    x.disabled = True
    y.disabled = True

In [22]:
def stat(x, y):
    print('Высота: {0}. Ширина: {1}.'.format(x, y))
    
interact(stat, x=x, y=y);
display(button1)
button1.on_click(on_button_clicked)

interactive(children=(IntSlider(value=2, description='Высота', max=40, min=2, step=2), FloatSlider(value=5.0, description='Ширина', max=80.0, min=2.0, step=0.5), Output()), _dom_classes=('widget-interact',))

Button(description='Нажмите, чтобы посчитать площадь прямоугольника!', layout=Layout(width='40%'), style=ButtonStyle())

Для полноты картины добавлю ещё одну кнопку, которая откроет данные для ввода, чтобы можно было ещё раз посчитать площать прямоугольника.

In [23]:
button2 = widgets.Button(description="Использовать другие данные", layout=widgets.Layout(width='40%'))

def on_button_clicked2(b):
    button1.disabled = False
    x.disabled = False
    y.disabled = False
    
display(button2)
button2.on_click(on_button_clicked2)

Button(description='Использовать другие данные', layout=Layout(width='40%'), style=ButtonStyle())

### Имитация оформления заказа на товар

На основе игрушечного примера о заказе компьютера я продемонстрирую большое количество разнообразных виджетов и то, как они могут быть использованы.

Для начала мы предлагаем пользователю выбрать интересующий его товар. Этот виджет позволяет выбрать одно значение из списка.

In [24]:
print('Выбор товара:')
pc_type = widgets.Select(
    options=['Компьютер', 'Планшет', 'Ноутбук'],
    value='Компьютер',
    description='Товар:'
)
pc_type

Выбор товара:


Select(description='Товар:', options=('Компьютер', 'Планшет', 'Ноутбук'), value='Компьютер')

Теперь предложим указать количество товара. `jslink` позволяет связать 2 виджета, для этого у них либо должны быть сравнимые значения (например числа), либо должно быть задано соответствие (например, при положительных значениях получается один текст, при неотрицательных - другой).

Также здесь используется новый виджет - BoundedIntText. Это поле для ввода integer, для которого можно задать минимальные и максимальные значения.

In [25]:
a = widgets.BoundedIntText(min=1, max=5)
b = widgets.IntSlider(min=1, max=5)
print('Укажите количество - введите число или выберите нужное значение')
display(a, b)
mylink = widgets.jslink((a, 'value'), (b, 'value'))

Укажите количество - введите число или выберите нужное значение


BoundedIntText(value=1, max=5, min=1)

IntSlider(value=1, max=5, min=1)

В качестве шутки предложим выбрать мощность покупаемой техники. Для этого используется виджет-слайдер для которого можно добавить ранжированный список текстовых значений.

In [26]:
power = widgets.SelectionSlider(
    options=['слабый', 'хороший', 'мощный', 'deep learning'],
    value='хороший',
    description='Мощность:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True, layout=widgets.Layout(width='40%')
)
power

SelectionSlider(continuous_update=False, description='Мощность:', index=1, layout=Layout(width='40%'), options=('слабый', 'хороший', 'мощный', 'deep learning'), value='хороший')

А теперь подарим клиенту бонус. Это виджет для множественного выбора с помощью клавиш Ctrl/Shift.

In [27]:
sel_mult = widgets.SelectMultiple(
    options=['Ручка', 'Стикер', 'Скидка'],
    value=['Стикер'],
    #rows=10,
    description='Бонусы',
    disabled=False
)
sel_mult

SelectMultiple(description='Бонусы', index=(1,), options=('Ручка', 'Стикер', 'Скидка'), value=('Стикер',))

Заказ укомплектован, теперь пора обговорить условия доставки. Первое - как будет доставлен товар. RadioButtons как и Select дают возможность выбора одной опции.

А теперь познакомимся с такой удобной штукой как HBox. Он позволяет делать более гибкую настройку отображения виджетов и их описания. Например, ширина описания виджетов ограничена, использовать Box изменения ширины довольно удобно. HBox и VBox являются соответственно горизонтальными и вертикальными контейнерами для виджетов или их описаний.

In [28]:
delivery = widgets.RadioButtons(
    options=['Доставка', 'Самовывоз'],
    disabled=False
)
widgets.HBox([widgets.Label(value="Способ доставки:"), delivery])

HBox(children=(Label(value='Способ доставки:'), RadioButtons(options=('Доставка', 'Самовывоз'), value='Доставка')))

Настала пора выбрать способ оплаты. ToggleButtons выводит кнопки, одна из которых может быть нажата. Виджет позволяет настроить названия кнопок, всплывающие тултипы и даже иконки (для задания иконок используются названия иконок для Font Awesome).

Также здесь используется задание параметров виджета используя style. Это дополнительный способ (по отношению к layout), но это уже не css, а внутренние свойства виджетов. Есть ряд свойств, которые относятся только к определённым виджетам.

In [29]:
pay_method = widgets.ToggleButtons(
    options=['Карта', 'Наличные'],
    description='Способ оплаты:',
    disabled=False,
    button_style='success',
    tooltips=['Карта', 'Наличные'],
     icons=['credit-card', 'money']
)
pay_method.style.description_width='200px'
pay_method

ToggleButtons(button_style='success', description='Способ оплаты:', icons=['credit-card', 'money'], options=('Карта', 'Наличные'), style=ToggleButtonsStyle(description_width='200px'), tooltips=['Карта', 'Наличные'], value='Карта')

Ещё есть виджет для выбора даты доставки.

In [30]:
date = widgets.DatePicker(
    description='',
    disabled=False
)
date

widgets.VBox([widgets.Label(value="Дата доставки:"), date])

VBox(children=(Label(value='Дата доставки:'), DatePicker(value=None)))

Дата вполне может быть выбрана неверно. Допустим мы позволяет выбрать дату в промежутке от 2 до 7 дней от текущего. Сделаем проверку.

In [31]:
button3 = widgets.Button(description="Проверить дату", layout=widgets.Layout(width='40%'))

def on_button_clicked3(d):
    valid = widgets.Valid(
        value=False,
        description='Корректная дата!',
        )
    valid.style.description_width='200px'
    
    if date.value == None:
        valid.description='Некорректная дата!'
        display(valid)
        print('Дата доставки не указана.')
        
    elif (date.value - datetime.datetime.today().date()).days in range(2,8):
        valid.value = True
        display(valid)
    else:
        valid.description='Некорректная дата!'
        display(valid)
        print('Дата доставки должна быть в периоде 2-7 дней от текущей даты.')
    
display(button3)
button3.on_click(on_button_clicked3)

Button(description='Проверить дату', layout=Layout(width='40%'), style=ButtonStyle())

Теперь можно выбрать доставки. IntRangeSlider позволяет указать интервал.

In [32]:
period = widgets.IntRangeSlider(
    value=[9, 18],
    min=7,
    max=22,
    step=1,
    description='Время:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='',
)
period

IntRangeSlider(value=(9, 18), continuous_update=False, description='Время:', max=22, min=7, orientation='vertical', readout_format='')

И, конечно, есть виджет для ввода текста. Для ввода короткого текста можно использовать Text, для текста побольше - Textarea.

И здесь мы видим событие on_submit при вводе текста - оно происходит после ввода текста и нажатия Enter.

In [33]:
text_comment = widgets.Text(
    value=' ',
    description='Комментарий к доставке:',
    disabled=False
)
text_comment

def text_submit(t=text_comment):
    print("Комментарий '{0}' зафиксирован.".format(text_comment.value.strip()))
    
text_comment.style.description_width='200px'
display(text_comment)
text_comment.on_submit(text_submit)

Text(value=' ', description='Комментарий к доставке:', style=DescriptionStyle(description_width='200px'))

Заказ оформлен, теперь надо бы показать клиенту все, что он указывал раньше, для подтверждения и предложить ввести адрес доставки. Первый вариант этого - сделать вложенные VBox и HBox, второй вариант - использовать Accordion и Tab.

Предварительно я "поворачиваю" слайдер для выбора временного периода доставки в горизонтальное положение, а также создаю новый виджет для ввода адреса доставки.

Теперь создам виджет Tab, который будет содержать Accordion. Accordion, в свою очередь, содержит в себе обычные виджеты, каждый из которых хранится на отдельной мини-вкладке. Это осуществляется перез параметр `children`. Названия этих мини-вкладок задаются для соответствующих индексов через `set_title`. Подобным образом Accordion вкладывается в Tab.

In [34]:
period.orientation='horizontal'
address = widgets.Textarea(
    value='Адрес',
    placeholder='Не дом и не улица',
    description='Адрес:',
    disabled=False
)

tab_nest = widgets.Tab()
accordion1 = widgets.Accordion(children=[pc_type, b, power, sel_mult])
accordion1.set_title(0, 'Товар:')
accordion1.set_title(1, 'Количество:')
accordion1.set_title(2, 'Мощность:')
accordion1.set_title(3, 'Бонус:')


accordion2 = widgets.Accordion(children=[widgets.HBox([widgets.Label(value="Способ доставки:"), delivery]),
                                        pay_method,
                                        widgets.VBox([widgets.Label(value="Дата доставки:"), date]),
                                        period,
                                        text_comment, address])
accordion2.set_title(0, 'Способ доставки:')
accordion2.set_title(1, 'Способ оплаты:')
accordion2.set_title(2, 'Дата доставки:')
accordion2.set_title(3, 'Время доставки:')
accordion2.set_title(4, 'Комментарий:')
accordion2.set_title(5, 'Адрес доставки:')


tab_nest.children = [accordion1, accordion2]
tab_nest.set_title(0, 'Заказ:')
tab_nest.set_title(1, 'Доставка:')
tab_nest

Tab(children=(Accordion(children=(Select(description='Товар:', options=('Компьютер', 'Планшет', 'Ноутбук'), value='Компьютер'), IntSlider(value=1, max=5, min=1), SelectionSlider(continuous_update=False, description='Мощность:', index=1, layout=Layout(width='40%'), options=('слабый', 'хороший', 'мощный', 'deep learning'), value='хороший'), SelectMultiple(description='Бонусы', index=(1,), options=('Ручка', 'Стикер', 'Скидка'), value=('Стикер',))), _titles={'0': 'Товар:', '1': 'Количество:', '2': 'Мощность:', '3': 'Бонус:'}), Accordion(children=(HBox(children=(Label(value='Способ доставки:'), RadioButtons(options=('Доставка', 'Самовывоз'), value='Доставка'))), ToggleButtons(button_style='success', description='Способ оплаты:', icons=['credit-card', 'money'], options=('Карта', 'Наличные'), style=ToggleButtonsStyle(description_width='200px'), tooltips=['Карта', 'Наличные'], value='Карта'), VBox(children=(Label(value='Дата доставки:'), DatePicker(value=None))), IntRangeSlider(value=(9, 18), continuous_update=False, description='Время:', max=22, min=7, readout_format=''), Text(value=' ', description='Комментарий к доставке:', style=DescriptionStyle(description_width='200px')), Textarea(value='Адрес', description='Адрес:', placeholder='Не дом и не улица')), _titles={'0': 'Способ доставки:', '1': 'Способ оплаты:', '2': 'Дата доставки:', '3': 'Время доставки:', '4': 'Комментарий:', '5': 'Адрес доставки:'})), _titles={'0': 'Заказ:', '1': 'Доставка:'})

На завершающем этапе видит итоговую информацию о заказе.
Во-первых, используется виджет для отображения прогресса. Самый удобный способ обновлять его значение - увеличивать его `value` при определенном событии.

In [35]:
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='',
    bar_style='success',
    orientation='horizontal'
)

Ещё один интересный виджет - `Out`. Он отображает то, что передаётся в него: это может быть просто печать данных через `print`, rich text, media и другие вещи.

In [36]:
out = widgets.Output()
out

Output()

In [37]:
with out:
    display(widgets.VBox([widgets.Label(value="Проверка информации  заказе:"), progress]))
    print('Информация о заказе:')
    for i in range(len(accordion1.children)):
        if type(accordion1.children[i].value) != tuple:
            print(accordion1.get_title(i), accordion1.children[i].value)
            progress.value +=1
            time.sleep(0.5) 
        else:
            print(accordion1.get_title(i), ' '.join([j for j in accordion1.children[i].value]))
            progress.value +=1
            time.sleep(0.5) 
            #print(accordion1.get_title(i)), [print(j) for j in accordion1.children[i].value]
    for i, e in enumerate([delivery, pay_method, date, period, text_comment, address]):
        print(accordion2.get_title(i), e.value)
        progress.value +=1
        time.sleep(0.5) 

Естественно, стоит добавлять проверки на адекватность значений и на пустые значения, но гайд не об этом, а о возможностях виджетов.

Я описал почти все возможности и способы применения ipywidgets. Есть ещё некоторые, но они являются более глубокими и будут реже использоваться.

In [38]:
from IPython.display import Javascript, display
display(Javascript('IPython.notebook.execute_cell()'))

<IPython.core.display.Javascript object>

In [39]:
widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)

Checkbox(value=False, description='Check me')

In [40]:
b1 = widgets.Button(description='Custom color')
b1.style.button_color = 'lightgreen'
b1

Button(description='Custom color', style=ButtonStyle(button_color='lightgreen'))

In [41]:
s1 = widgets.IntSlider(description='Blue handle')
s1.style.handle_color = 'lightblue'
s1

IntSlider(value=0, description='Blue handle', style=SliderStyle(handle_color='lightblue'))

In [48]:
from ipywidgets import StaticInteract, RangeWidget, RadioWidget

def multiply(a, b):
    return "{0} * {1} = {2}".format(a, b, a * b)

StaticInteract(multiply,
               a=RangeWidget(0, 10),
               b=RangeWidget(0, 10),)

ImportError: cannot import name 'StaticInteract'

In [42]:
from sklearn.datasets import load_iris

In [43]:
iris = load_iris()
X = iris.data
y = iris.target

In [44]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [47]:
def f(x):
    return x
interact(f, x=iris.feature_names);

interactive(children=(Dropdown(description='x', options=('sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'), value='sepal length (cm)'), Output()), _dom_classes=('widget-interact',))

* https://machinelearningmastery.com/machine-learning-in-python-step-by-step/
* http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html
* https://stackoverflow.com/questions/37013489/how-to-alight-and-place-ipywidgets